# Fluidity Model: Flow Curves and Thixotropic Yielding

## Learning Objectives

1. Fit the **Fluidity model** to emulsion flow curve data
2. Understand the structure kinetics: $\frac{df}{dt} = \frac{f_{eq} - f}{\theta} + a|\dot{\gamma}|^{n_{rejuv}}(f_{\infty} - f)$
3. Predict Herschel-Bulkley flow behavior: $\sigma = \tau_y + K|\dot{\gamma}|^n$
4. Analyze volume fraction dependence of yield stress and fluidity evolution
5. Use NLSQ warm-start for efficient Bayesian inference with NUTS
6. Extract posterior distributions for 9 model parameters

## Prerequisites

- Basic RheoJAX usage (basic/01_quick_start.ipynb)
- Bayesian inference fundamentals (bayesian/01_bayesian_basics.ipynb)

## Runtime

- Fast demo (NUM_CHAINS=1, NUM_SAMPLES=500): ~2-3 minutes
- Full run (NUM_CHAINS=4, NUM_SAMPLES=2000): ~8-10 minutes

## 1. Setup

In [ ]:
# Google Colab setup
import sys

IN_COLAB = "google.colab" in sys.modules
if IN_COLAB:
    %pip install -q rheojax
    import os
    os.environ["JAX_ENABLE_X64"] = "true"
    print("RheoJAX installed successfully.")

In [ ]:
# Imports
%matplotlib inline
import json
import os
import sys
import time
import warnings

import arviz as az
import matplotlib.pyplot as plt
import numpy as np
from IPython.display import display

from rheojax.core.jax_config import safe_import_jax, verify_float64
from rheojax.models.fluidity import FluidityLocal

# Add examples/utils to path for tutorial utilities
sys.path.insert(0, os.path.join("..", "utils"))
from fluidity_tutorial_utils import (
    load_emulsion_flow_curve,
    save_fluidity_results,
    print_convergence_summary,
    print_parameter_comparison,
)

jax, jnp = safe_import_jax()
verify_float64()

warnings.filterwarnings("ignore", category=FutureWarning)
print(f"JAX version: {jax.__version__}")
print(f"Devices: {jax.devices()}")

In [ ]:
def compute_fit_quality(y_true, y_pred):
    """Compute R² and RMSE."""
    y_true = np.asarray(y_true)
    y_pred = np.asarray(y_pred)
    residuals = y_true - y_pred
    if y_true.ndim > 1:
        residuals = residuals.ravel()
        y_true = y_true.ravel()
    ss_res = np.sum(residuals**2)
    ss_tot = np.sum((y_true - np.mean(y_true))**2)
    r2 = 1.0 - ss_res / ss_tot if ss_tot > 0 else 0.0
    rmse = np.sqrt(np.mean(residuals**2))
    return {"R2": r2, "RMSE": rmse}

## 2. Theory: Fluidity Model

The **Fluidity model** describes yield-stress fluids through a **structure parameter** $f$ (fluidity = 1/viscosity) that evolves according to aging and flow-induced rejuvenation.

### Key Equations

**Structure kinetics** (thixotropic fluidity evolution):
$$
\frac{df}{dt} = \frac{f_{eq} - f}{\theta} + a|\dot{\gamma}|^{n_{rejuv}}(f_{\infty} - f)
$$

**Stress evolution** (Maxwell-like viscoelasticity):
$$
\frac{d\sigma}{dt} = G\left(\dot{\gamma} - \sigma f\right)
$$

**Steady-state fluidity**:
$$
f_{ss} = \frac{f_{eq}/\theta + a|\dot{\gamma}|^{n_{rejuv}} f_{\infty}}{1/\theta + a|\dot{\gamma}|^{n_{rejuv}}}
$$

**Steady-state stress** (Herschel-Bulkley flow curve):
$$
\sigma_{ss} = \frac{\dot{\gamma}}{f_{ss}} = \tau_y + K|\dot{\gamma}|^n
$$

### Parameters (9 total)

| Parameter | Symbol | Physical Meaning | Typical Range |
|-----------|--------|------------------|---------------|
| **G** | $G$ | Elastic modulus (Pa) | 10–10000 Pa |
| **tau_y** | $\tau_y$ | Yield stress (Pa) | 1–1000 Pa |
| **K** | $K$ | Flow consistency (Pa·s$^n$) | 1–100 Pa·s$^n$ |
| **n_flow** | $n$ | Flow exponent (dimensionless) | 0.3–1.0 |
| **f_eq** | $f_{eq}$ | Equilibrium fluidity (1/(Pa·s)) | 0.001–0.1 1/(Pa·s) |
| **f_inf** | $f_{\infty}$ | High-shear fluidity (1/(Pa·s)) | 0.01–1.0 1/(Pa·s) |
| **theta** | $\theta$ | Structural relaxation time (s) | 1–1000 s |
| **a** | $a$ | Rejuvenation amplitude | 0.1–10 |
| **n_rejuv** | $n_{rejuv}$ | Rejuvenation exponent | 0.5–2.0 |

### Physical Interpretation

- **Aging** ($f_{eq}/\theta$): At rest, fluidity decays to equilibrium value $f_{eq}$ (material stiffens)
- **Rejuvenation** ($a|\dot{\gamma}|^{n_{rejuv}}$): Flow breaks structure, increasing fluidity toward $f_{\infty}$
- **Yield stress**: Emerges from low equilibrium fluidity $f_{eq} \ll f_{\infty}$
- **Shear-thinning**: Flow exponent $n < 1$ indicates rate-dependent viscosity decrease

## 3. Load Data

We use six emulsion datasets at different volume fractions ($\phi = 0.69, 0.70, 0.72, 0.74, 0.76, 0.80$). Each CSV contains (Shear Rate, Stress) columns.

**Note:** The Fluidity model predicts **stress** $\sigma(\dot{\gamma})$ directly, unlike SGR which predicts viscosity.

In [ ]:
# Load all 6 emulsion datasets
phi_values = [0.69, 0.70, 0.72, 0.74, 0.76, 0.80]
datasets = {}

for phi in phi_values:
    gamma_dot, stress = load_emulsion_flow_curve(phi)
    datasets[phi] = {"gamma_dot": gamma_dot, "stress": stress}
    print(f"φ={phi:.2f}: {len(gamma_dot)} points, γ̇=[{gamma_dot.min():.4f}, {gamma_dot.max():.1f}] 1/s")

In [ ]:
# Plot all 6 flow curves
fig, ax = plt.subplots(figsize=(10, 6))
colors = plt.cm.viridis(np.linspace(0.2, 0.9, len(phi_values)))

for i, phi in enumerate(phi_values):
    d = datasets[phi]
    ax.loglog(d["gamma_dot"], d["stress"], "o", color=colors[i], markersize=5, label=f"φ={phi:.2f}")

ax.set_xlabel("Shear rate [1/s]", fontsize=12)
ax.set_ylabel("Stress [Pa]", fontsize=12)
ax.set_title("Emulsion Flow Curves (Raw Data)", fontsize=14)
ax.legend(fontsize=9)
ax.grid(True, alpha=0.3, which="both")
plt.tight_layout()
display(fig)
plt.close(fig)

## 4. NLSQ Fitting

### 4.1 Single Sample (φ=0.80)

We start by fitting the most concentrated emulsion ($\phi=0.80$), which should exhibit strong yield stress and shear-thinning behavior.

In [ ]:
# Fit FluidityLocal to φ=0.80 sample
model = FluidityLocal()

gamma_dot_80 = datasets[0.80]["gamma_dot"]
stress_80 = datasets[0.80]["stress"]

t0 = time.time()
model.fit(gamma_dot_80, stress_80, test_mode="flow_curve", method='scipy')
t_nlsq = time.time() - t0

print(f"NLSQ fit time: {t_nlsq:.2f} s")
# Compute metrics
stress_pred = model.predict(gamma_dot_80, test_mode="flow_curve")
metrics = compute_fit_quality(stress_80, stress_pred)

print(f"R²: {metrics['R2']:.6f}")
print(f"RMSE: {metrics['RMSE']:.4g} Pa")
print("\nFitted parameters:")
param_names = ["G", "tau_y", "K", "n_flow", "f_eq", "f_inf", "theta", "a", "n_rejuv"]
for name in param_names:
    val = model.parameters.get_value(name)
    print(f"  {name:10s} = {val:.4g}")

In [ ]:
# Plot fit with data
gamma_dot_fine = np.logspace(
    np.log10(gamma_dot_80.min()) - 0.5,
    np.log10(gamma_dot_80.max()) + 0.3,
    200,
)
stress_pred = model.predict(gamma_dot_fine)

fig, ax = plt.subplots(figsize=(9, 6))
ax.loglog(gamma_dot_80, stress_80, "ko", markersize=6, label="Data φ=0.80")
ax.loglog(gamma_dot_fine, stress_pred, "-", lw=2, color="C0", label="Fluidity fit")
ax.set_xlabel("Shear rate [1/s]", fontsize=12)
ax.set_ylabel("Stress [Pa]", fontsize=12)
tau_y = model.parameters.get_value("tau_y")
n_flow = model.parameters.get_value("n_flow")
ax.set_title(f"Fluidity Fit — τ_y={tau_y:.1f} Pa, n={n_flow:.3f}", fontsize=13)
ax.legend(fontsize=10)
ax.grid(True, alpha=0.3, which="both")
plt.tight_layout()
display(fig)
plt.close(fig)

### 4.2 Volume Fraction Sweep

Now we fit all six volume fractions to observe how the yield stress $\tau_y$ and flow exponent $n$ evolve as the emulsion approaches jamming.

In [ ]:
# Fit all 6 samples and track parameters
fit_results = {}

for phi in phi_values:
    m = FluidityLocal()
    d = datasets[phi]
    m.fit(d["gamma_dot"], d["stress"], test_mode="flow_curve", method='scipy')
    stress_pred = m.predict(d["gamma_dot"], test_mode="flow_curve")
    metrics = compute_fit_quality(d["stress"], stress_pred)
    
    fit_results[phi] = {
        "G": float(m.parameters.get_value("G")),
        "tau_y": float(m.parameters.get_value("tau_y")),
        "K": float(m.parameters.get_value("K")),
        "n_flow": float(m.parameters.get_value("n_flow")),
        "f_eq": float(m.parameters.get_value("f_eq")),
        "f_inf": float(m.parameters.get_value("f_inf")),
        "theta": float(m.parameters.get_value("theta")),
        "a": float(m.parameters.get_value("a")),
        "n_rejuv": float(m.parameters.get_value("n_rejuv")),
        "R2": metrics["R2"],
    }

print(f"{'φ':>6s}  {'τ_y':>8s}  {'K':>8s}  {'n':>6s}  {'f_eq':>10s}  {'f_∞':>10s}  {'θ':>10s}  {'R²':>8s}")
print("-" * 80)
for phi in phi_values:
    r = fit_results[phi]
    print(f"{phi:6.2f}  {r['tau_y']:8.2f}  {r['K']:8.2f}  {r['n_flow']:6.3f}  {r['f_eq']:10.4g}  {r['f_inf']:10.4g}  {r['theta']:10.2g}  {r['R2']:8.5f}")

In [ ]:
# Plot τ_y(φ) and n(φ) trends
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

phis = list(fit_results.keys())
tau_y_vals = [fit_results[p]["tau_y"] for p in phis]
n_vals = [fit_results[p]["n_flow"] for p in phis]

# Yield stress vs volume fraction
ax1.plot(phis, tau_y_vals, "o-", markersize=8, lw=2, color="C1")
ax1.set_xlabel("Volume fraction φ", fontsize=12)
ax1.set_ylabel("Yield stress τ_y [Pa]", fontsize=12)
ax1.set_title("Yield Stress vs Volume Fraction", fontsize=13)
ax1.grid(True, alpha=0.3)

# Flow exponent vs volume fraction
ax2.plot(phis, n_vals, "o-", markersize=8, lw=2, color="C2")
ax2.axhline(1.0, color="red", linestyle="--", alpha=0.5, label="Newtonian (n=1)")
ax2.set_xlabel("Volume fraction φ", fontsize=12)
ax2.set_ylabel("Flow exponent n", fontsize=12)
ax2.set_title("Shear-Thinning Exponent vs Volume Fraction", fontsize=13)
ax2.legend(fontsize=9)
ax2.grid(True, alpha=0.3)

plt.tight_layout()
display(fig)
plt.close(fig)

### 4.3 Fluidity Evolution Analysis

The equilibrium fluidity $f_{eq}$ and high-shear fluidity $f_{\infty}$ control the transition from solid-like (low $f_{eq}$) to liquid-like (high $f_{\infty}$) behavior.

In [ ]:
# Plot fluidity ratio f_inf/f_eq vs volume fraction
f_eq_vals = [fit_results[p]["f_eq"] for p in phis]
f_inf_vals = [fit_results[p]["f_inf"] for p in phis]
fluidity_ratio = [f_inf_vals[i] / f_eq_vals[i] for i in range(len(phis))]

fig, ax = plt.subplots(figsize=(9, 6))
ax.semilogy(phis, fluidity_ratio, "o-", markersize=8, lw=2, color="C3")
ax.set_xlabel("Volume fraction φ", fontsize=12)
ax.set_ylabel("Fluidity ratio $f_{\\infty}/f_{eq}$", fontsize=12)
ax.set_title("Flow-Induced Fluidity Enhancement", fontsize=13)
ax.grid(True, alpha=0.3)
ax.text(
    0.05, 0.95,
    "Higher ratio → stronger\nshear-induced restructuring",
    transform=ax.transAxes,
    fontsize=10,
    verticalalignment="top",
    bbox=dict(boxstyle="round", facecolor="wheat", alpha=0.3),
)
plt.tight_layout()
display(fig)
plt.close(fig)

### 4.4 Fit Quality Assessment

Check R² across all volume fractions to validate model applicability.

In [ ]:
# Plot R² vs volume fraction
r2_vals = [fit_results[p]["R2"] for p in phis]

fig, ax = plt.subplots(figsize=(9, 6))
ax.bar(phis, r2_vals, width=0.015, color="C0", alpha=0.7, edgecolor="black")
ax.set_xlabel("Volume fraction φ", fontsize=12)
ax.set_ylabel("R²", fontsize=12)
ax.set_title("Fit Quality Across Volume Fractions", fontsize=13)
ax.set_ylim(0.95, 1.005)
ax.axhline(0.99, color="green", linestyle="--", alpha=0.5, label="Excellent (R²>0.99)")
ax.legend(fontsize=9)
ax.grid(True, alpha=0.3, axis="y")
plt.tight_layout()
display(fig)
plt.close(fig)

## 5. Bayesian Inference with NUTS

### 5.1 Run NUTS

We use the NLSQ point estimates as warm-start values for efficient MCMC sampling with the No-U-Turn Sampler (NUTS).

In [ ]:
# Bayesian inference with NLSQ warm-start
initial_values = {
    name: model.parameters.get_value(name)
    for name in param_names
}
print("Warm-start values:")
for name, val in initial_values.items():
    print(f"  {name:10s} = {val:.4g}")

# Fast demo config
NUM_WARMUP = 200
NUM_SAMPLES = 500
NUM_CHAINS = 1
# NUM_WARMUP = 1000; NUM_SAMPLES = 2000; NUM_CHAINS = 4  # production

print(f"\nRunning NUTS: {NUM_WARMUP} warmup + {NUM_SAMPLES} samples × {NUM_CHAINS} chain(s)")
t0 = time.time()
result = model.fit_bayesian(
    gamma_dot_80,
    stress_80,
    test_mode="flow_curve",
    num_warmup=NUM_WARMUP,
    num_samples=NUM_SAMPLES,
    num_chains=NUM_CHAINS,
    initial_values=initial_values,
    seed=42,
)
t_bayes = time.time() - t0
print(f"\nBayesian inference time: {t_bayes:.1f} s")

### 5.2 Convergence Diagnostics

Check R-hat (target < 1.05) and effective sample size (ESS, target > 100 for demo).

In [ ]:
# Convergence diagnostics table
all_pass = print_convergence_summary(result, param_names)

### 5.3 ArviZ Diagnostic Plots

In [ ]:
# Trace plots
idata = result.to_inference_data()

# Plot subset of key parameters for clarity
key_params = ["tau_y", "K", "n_flow", "f_eq", "f_inf", "theta"]
axes = az.plot_trace(idata, var_names=key_params, figsize=(12, 8))
fig = axes.ravel()[0].figure
fig.suptitle("Trace Plots (Key Parameters)", fontsize=14, y=1.00)
plt.tight_layout()
display(fig)
plt.close(fig)

In [ ]:
# Pair plot for physical parameter correlations
phys_params = ["tau_y", "K", "n_flow", "theta"]
axes = az.plot_pair(
    idata,
    var_names=phys_params,
    kind="scatter",
    divergences=True,
    figsize=(10, 10),
)
fig = axes.ravel()[0].figure
fig.suptitle("Physical Parameter Correlations", fontsize=14, y=1.00)
plt.tight_layout()
display(fig)
plt.close(fig)

In [ ]:
# Forest plot (credible intervals) for all parameters
axes = az.plot_forest(
    idata,
    var_names=param_names,
    combined=True,
    hdi_prob=0.95,
    figsize=(10, 6),
)
fig = axes.ravel()[0].figure
fig.suptitle("95% Credible Intervals (All Parameters)", fontsize=13)
plt.tight_layout()
display(fig)
plt.close(fig)

### 5.4 Posterior Predictive Check

Validate the model by comparing data against predictions from posterior samples.

In [ ]:
# Posterior predictive with 95% credible intervals
posterior = result.posterior_samples
n_draws = min(200, len(list(posterior.values())[0]))
gamma_dot_pred = np.logspace(
    np.log10(gamma_dot_80.min()) - 0.3,
    np.log10(gamma_dot_80.max()) + 0.2,
    100,
)

pred_samples = []
for i in range(n_draws):
    params_i = jnp.array([posterior[name][i] for name in param_names])
    pred_i = model.model_function(jnp.array(gamma_dot_pred), params_i, test_mode="flow_curve")
    pred_samples.append(np.array(pred_i))

pred_samples = np.array(pred_samples)
pred_median = np.median(pred_samples, axis=0)
pred_lo = np.percentile(pred_samples, 2.5, axis=0)
pred_hi = np.percentile(pred_samples, 97.5, axis=0)

fig, ax = plt.subplots(figsize=(10, 6))
ax.fill_between(gamma_dot_pred, pred_lo, pred_hi, alpha=0.3, color="C0", label="95% CI")
ax.loglog(gamma_dot_pred, pred_median, "-", lw=2, color="C0", label="Posterior median")
ax.loglog(gamma_dot_80, stress_80, "ko", markersize=6, label="Data")
ax.set_xlabel("Shear rate [1/s]", fontsize=12)
ax.set_ylabel("Stress [Pa]", fontsize=12)
ax.set_title("Posterior Predictive Check (φ=0.80)", fontsize=13)
ax.legend(fontsize=10)
ax.grid(True, alpha=0.3, which="both")
plt.tight_layout()
display(fig)
plt.close(fig)

### 5.5 Parameter Summary

Compare NLSQ point estimates with Bayesian posterior medians and 95% credible intervals.

In [ ]:
# NLSQ vs Bayesian comparison table
print_parameter_comparison(model, posterior, param_names)

## 6. Physical Interpretation

### Herschel-Bulkley Flow Curve

The Fluidity model recovers the classic **Herschel-Bulkley** constitutive equation at steady state:
$$
\sigma(\dot{\gamma}) = \tau_y + K|\dot{\gamma}|^n
$$

where:
- **$\tau_y$** (yield stress): Minimum stress required to initiate flow
- **$K$** (consistency): Material stiffness at unit shear rate
- **$n$** (flow exponent): $n < 1$ → shear-thinning, $n = 1$ → Bingham plastic

### Thixotropic Dynamics

The time-dependent structure parameter $f(t)$ enables transient predictions:
- **Startup overshoot**: Solid-like structure ($f_{eq}$) breaks down under flow
- **Creep**: Delayed yielding as fluidity increases from $f_{eq}$ → $f_{\infty}$
- **Relaxation**: Partial recovery of structure after flow cessation

### Volume Fraction Trends

- **Increasing $\phi$**: Yield stress $\tau_y$ grows rapidly near jamming transition ($\phi_c \approx 0.64$)
- **Flow exponent $n$**: Decreases with $\phi$ (stronger shear-thinning in denser emulsions)
- **Fluidity ratio $f_{\infty}/f_{eq}$**: Higher in concentrated samples (more dramatic flow-induced restructuring)

## 7. Save Results

In [ ]:
# Save results for downstream notebooks
save_fluidity_results(model, result, "local", "flow_curve", param_names)

# Save volume fraction sweep results
output_dir = os.path.join("..", "outputs", "fluidity", "local", "flow_curve")
os.makedirs(output_dir, exist_ok=True)

with open(os.path.join(output_dir, "phi_sweep_results.json"), "w") as f:
    json.dump({str(k): v for k, v in fit_results.items()}, f, indent=2)

print(f"\nAdditional results saved:")
print(f"  phi_sweep_results.json: {len(fit_results)} volume fractions")

## Key Takeaways

1. **Fluidity model predicts flow curves** via Herschel-Bulkley equation: $\sigma = \tau_y + K|\dot{\gamma}|^n$

2. **Structure kinetics** control thixotropic behavior:
   - **Aging** ($f_{eq}/\theta$): Material stiffens at rest
   - **Rejuvenation** ($a|\dot{\gamma}|^{n_{rejuv}}$): Flow breaks structure, increasing fluidity

3. **Volume fraction trends** near jamming:
   - Yield stress $\tau_y$ increases sharply with $\phi$
   - Flow exponent $n$ decreases (stronger shear-thinning)
   - Fluidity ratio $f_{\infty}/f_{eq}$ grows (more dramatic restructuring)

4. **NLSQ warm-start is critical** for efficient Bayesian inference:
   - 9 parameters require good initial values for NUTS convergence
   - R-hat < 1.05 and ESS > 100 achieved in demo mode

5. **High fit quality** (R² > 0.99) across all volume fractions validates model applicability to emulsion rheology

6. **Parameter correlations** (pair plot) reveal physical dependencies:
   - $\tau_y$ ↔ $f_{eq}$ (low equilibrium fluidity → high yield stress)
   - $K$ ↔ $n$ (consistency and shear-thinning linked)

### Next Steps

- **NB 02**: Startup transients (stress overshoot from structure breakdown)
- **NB 03**: Creep compliance (delayed yielding and viscosity bifurcation)
- **NB 04**: Stress relaxation (partial structure recovery)
- **NB 05**: SAOS moduli (linear viscoelasticity)
- **NB 06**: LAOS harmonics (nonlinear oscillatory response)